# Phase 1 : 24-08-2025 13:13:04

Building and testing basic AES, DES Encryption

In [1]:
# @title 🧰 Setup and Installation for Phase 1

!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.5 MB/s eta 0:00:00


In [2]:
# @title 🔐 AES and DES Encryption and Decryption Module Creation

from Crypto.Cipher import AES, DES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return cipher.iv + ct_bytes

def aes_decrypt(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def des_encrypt(plaintext, key):
    cipher = DES.new(key, DES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext.encode(), DES.block_size))
    return cipher.iv + ct_bytes

def des_decrypt(ciphertext, key):
    iv = ciphertext[:8]
    ct = ciphertext[8:]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), DES.block_size)
    return pt.decode()

def encrypt_with_algo(plaintext, key, algorithm='AES'):
    if algorithm == 'AES':
        return aes_encrypt(plaintext, key)
    elif algorithm == 'DES':
        return des_encrypt(plaintext, key)
    else:
        raise ValueError("Unsupported algorithm")

def decrypt_with_algo(ciphertext, key, algorithm='AES'):
    if algorithm == 'AES':
        return aes_decrypt(ciphertext, key)
    elif algorithm == 'DES':
        return des_decrypt(ciphertext, key)
    else:
        raise ValueError("Unsupported algorithm")

In [3]:
# @title Testing Phase 1

# pt = input("Enter a text for encryption : ")
# key = get_random_bytes(16)
# ct_aes = encrypt_with_algo(pt, key, 'AES')
# print("AES Encryption : ", ct_aes.hex())
# print("AES Decryption : ", decrypt_with_algo(ct_aes, key, 'AES'))

# key = get_random_bytes(8)
# ct_des = encrypt_with_algo(pt, key, 'DES')
# print("DES Encryption : ", ct_des.hex())
# print("DES Decryption : ", decrypt_with_algo(ct_des, key, 'DES'))

# Phase 2 : 24-08-2025 14:21:24

Building and testing Quantum Key using QRNG

In [4]:
# @title 🧰 Setup and Installation for Phase 2

!pip install qiskit qiskit-aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.1/126.1 kB 9.9 MB/s eta 0:00:00


In [5]:
# @title 🔑 Generate Random Quantum Key

from qiskit import QuantumCircuit
from qiskit_aer.primitives import Sampler
import numpy as np

def generate_quantum_key(bits=128):
    qc = QuantumCircuit(bits, bits)
    qc.h(range(bits))
    qc.measure(range(bits), range(bits))

    # Use Aer Sampler to simulate and get outcome
    sampler = Sampler()
    result = sampler.run(qc).result()
    counts = result.quasi_dists[0]

    # Get the most probable bitstring
    max_key = max(counts, key=counts.get)
    bit_str = bin(max_key)[2:]

    # Pad with leading zeros if necessary
    if len(bit_str) < bits:
        bit_str = '0' * (bits - len(bit_str)) + bit_str

    # Convert to byte key
    key_bytes = bytes(int(bit_str[i:i+8], 2) for i in range(0, bits, 8))
    return key_bytes

In [6]:
# @title 🧪 Test

aes_key = generate_quantum_key(128)
des_key = generate_quantum_key(64)
# print("Quantum AES Key:", key.hex(), "| Length:", len(key))

/tmp/ipython-input-3945919019.py:3: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  aes_key = generate_quantum_key(128)
/tmp/ipython-input-3945919019.py:4: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  des_key = generate_quantum_key(64)


In [7]:
# @title ✅ Using Generated keys for AES and DES

plaintext = "I am unstoppable"

aes_ct = aes_encrypt(plaintext, aes_key)
print("AES Key :", aes_key.hex())
print("AES Encrypted :", aes_ct.hex())
print("AES Decrypted :", aes_decrypt(aes_ct, aes_key))

des_ct = des_encrypt(plaintext, des_key)
print("dES Key :", des_key.hex())
print("dES Encrypted :", des_ct.hex())
print("dES Decrypted :", des_decrypt(des_ct, des_key))

AES Key : 5afe2c4d3b244a2f84852d718051f786
AES Encrypted : 3cf5641aded8f4986222a9839e6eab6a13091b688fe01ba139d94bbc62c87701db48f40736a3afc25b02115fb71dc9d1
AES Decrypted : I am unstoppable
dES Key : b3304ba279a79ea3
dES Encrypted : 699e738cb65cfee63dde9cbe8525b0f0b8a41fffe4502e4eed2f446c3b885b8a
dES Decrypted : I am unstoppable


# Phase 3 : 24-08-2025 14:47:11

Simulating BB84 protocol for secure key agreement

In [8]:
# @title Basic 🅱️🅱️𝟾𝟺 Code

import numpy as np

def random_bitstring(length):
    return np.random.randint(2, size=length)

def random_bases(length):
    return np.random.randint(2, size=length)

def encode_qubits(bits, bases):
    encoded = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if basis == 1:
            qc.h(0)
        encoded.append(qc)
    return encoded

In [9]:
# @title ↔️ Transmission attempt

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile

def measure_qubits(qubits, bases, eve_present=False):
    backend = AerSimulator()
    measurements = []

    for i, qc in enumerate(qubits):
        circuit = qc.copy()

        if eve_present:
            eve_basis = np.random.randint(2)
            if eve_basis == 1:
                circuit.h(0)
            circuit.measure(0, 0)

            transpiled = transpile(circuit, backend)
            result = backend.run(transpiled, shots=1, memory=True).result()
            intercepted_bit = int(result.get_memory()[0])

            circuit = QuantumCircuit(1, 1)
            if intercepted_bit == 1:
                circuit.x(0)
            if bases[i] == 1:
                circuit.h(0)

        if bases[i] == 1:
            circuit.h(0)
        circuit.measure(0, 0)

        transpiled = transpile(circuit, backend)
        result = backend.run(transpiled, shots=1, memory=True).result()
        bit = int(result.get_memory()[0])
        measurements.append(bit)

    return measurements

In [10]:
# @title Key Sifting and Comparison

def sift_keys(alice_bits, alice_bases, bob_bits, bob_bases):
    sifted_alice = []
    sifted_bob = []
    for i in range(len(alice_bits)):
        if alice_bases[i] == bob_bases[i]:
            sifted_alice.append(alice_bits[i])
            sifted_bob.append(bob_bits[i])

    return [int(x) for x in sifted_alice], sifted_bob

In [11]:
# @title 🎓 Full Demo

def demo(eve = False, n_bits=256, min_key_len=10):
    alice_bits = random_bitstring(n_bits)
    alice_bases = random_bases(n_bits)
    bob_bases = random_bases(n_bits)

    qubits = encode_qubits(alice_bits, alice_bases)

    bob_bits = measure_qubits(qubits, bob_bases, eve)

    alice_key, bob_key = sift_keys(alice_bits, alice_bases, bob_bits, bob_bases)

    # print("Alice Key:", alice_key)
    # print("Bob Key:  ", bob_key)

    key_len = len(alice_key)

    if key_len < min_key_len:
        # signal caller to retry this trial (too little data to judge)
        return None  # meaning "skip"

    error_bits = sum(a != b for a, b in zip(alice_key, bob_key))
    error_rate = error_bits / key_len

    # print(f"\nKey Length: {len(alice_key)}")
    # print(f"Error Rate: {error_rate:.2%} {'❌ (Eve detected)' if error_rate > 0.25 else '✅ (Secure)'}")

    return error_rate

In [12]:
# @title 📝 Repeated testing with threshold + retries

# import random

# trials = int(input("Enter number of trials : "))
# n_bits = 100                 # more stable than 50
# threshold = 0.15             # 0.25 theoretical; margin for finite-sample noise
# min_key_len = 10
# max_retries_per_trial = 3    # in case sifted key is too short

# true_eve = false_eve = false_not_eve = true_not_eve = 0
# skipped = 0

# for i in range(trials):
#     # Try to get a usable trial (enough sifted bits)
#     print("Starting ", i)
#     for attempt in range(max_retries_per_trial):
#         eve = random.choice([True, False])
#         error_rate = demo(eve=eve, n_bits=n_bits, min_key_len=min_key_len)
#         if error_rate is not None:
#             break
#     else:
#         skipped += 1
#         print("Trying ", i, " ", attempt, " time.")
#         continue  # couldn’t get a usable key; skip this trial

#     detected = (error_rate >= threshold)

#     if eve and detected:
#         true_eve += 1
#     elif eve and not detected:
#         false_eve += 1
#     elif (not eve) and detected:
#         false_not_eve += 1
#     else:
#         true_not_eve += 1

# print("Eve present & correctly flagged (TP):  ", true_eve)
# print("Eve present & missed (FN):             ", false_eve)
# print("No Eve but flagged (FP):               ", false_not_eve)
# print("No Eve & not flagged (TN):             ", true_not_eve)
# print("Skipped trials (short keys):           ", skipped)

# total = true_eve + false_eve + false_not_eve + true_not_eve
# if total > 0:
#     accuracy = round((true_eve + true_not_eve) / total * 100, 2)
#     print("Correct Identification: ", accuracy, "%")
# else:
#     print("No usable trials (all skipped). Try increasing n_bits or retries.")

# Phase 4 : 24-08-2025 21:14:22

Encryption and Decryption using quantum keys as declared in Phase 3 with AES and DES modules in Phase 1

In [13]:
# @title 😣 BB84 Helpers

def random_bitstring(n):
    return np.random.randint(2, size=n).tolist()

def random_bases(n):      # 0 = Z basis, 1 = X basis
    return np.random.randint(2, size=n).tolist()

def encode_qubits(bits, bases):
    """Build one-qubit circuits per bit with chosen basis."""
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:     # prepare |1>
            qc.x(0)
        if basis == 1:   # switch to X basis (|+>, |->)
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_qubits(qubits, bob_bases, eve_present=False):
    """
    Bob measures each received qubit in his chosen basis.
    Optional Eve: intercepts (random basis), measures, then resends in HER basis,
    which statistically induces ~25% errors after sifting.
    """
    backend = AerSimulator()
    out_bits = []

    for i, prep in enumerate(qubits):
        circuit = prep.copy()

        if eve_present:
            # Eve chooses a random basis and measures
            eve_basis = np.random.randint(2)
            eve_circ = circuit.copy()
            if eve_circ.num_clbits == 0:
                eve_circ.add_register(*QuantumCircuit(1,1).cregs)
            if eve_basis == 1:
                eve_circ.h(0)
            eve_circ.measure(0, 0)
            tr = transpile(eve_circ, backend)
            res = backend.run(tr, shots=1, memory=True).result()
            intercepted = int(res.get_memory()[0])

            # Eve resends in the basis she measured
            circuit = QuantumCircuit(1, 1)
            if intercepted == 1: circuit.x(0)
            if eve_basis == 1:    circuit.h(0)

        # Bob applies his basis and measures
        if circuit.num_clbits == 0:
            circuit.add_register(*QuantumCircuit(1,1).cregs)
        if bob_bases[i] == 1:
            circuit.h(0)
        circuit.measure(0, 0)

        trb = transpile(circuit, backend)
        resb = backend.run(trb, shots=1, memory=True).result()
        bit = int(resb.get_memory()[0])
        out_bits.append(bit)

    return out_bits

def sift_keys(alice_bits, alice_bases, bob_bits, bob_bases):
    """Keep positions where bases match."""
    idx = [i for i in range(len(alice_bits)) if alice_bases[i] == bob_bases[i]]
    a_sift = [alice_bits[i] for i in idx]
    b_sift = [bob_bits[i]  for i in idx]
    return a_sift, b_sift, idx

In [14]:
# @title 🗝️ Short Key Handling

def get_bb84_key(target_bits=128, n_per_round=256, qber_cutoff=0.11,
                 max_rounds=20, eve_present=False, verbose=True):
    """
    Repeats BB84 rounds until we collect at least `target_bits` sifted, low-QBER bits.
    - qber_cutoff ~ 11% is a practical threshold (abort round if higher).
    - If Eve is present (True), rounds will be discarded due to high QBER.
    """
    collected = []
    last_qber = None

    for r in range(1, max_rounds+1):
        # Alice & Bob choose bits/bases
        alice_bits  = random_bitstring(n_per_round)
        alice_bases = random_bases(n_per_round)
        bob_bases   = random_bases(n_per_round)

        # Alice encodes; channel → Bob measures (optionally with Eve)
        qubits  = encode_qubits(alice_bits, alice_bases)
        bob_bits = measure_qubits(qubits, bob_bases, eve_present=eve_present)

        # Sift and compute QBER (exact here; in practice you'd sample)
        a_sift, b_sift, _ = sift_keys(alice_bits, alice_bases, bob_bits, bob_bases)
        if len(a_sift) == 0:
            if verbose: print(f"Round {r}: no matches (unlikely).")
            continue
        errors = sum(int(a != b) for a, b in zip(a_sift, b_sift))
        qber = errors / len(a_sift)
        last_qber = qber

        if verbose:
            print(f"Round {r}: sifted={len(a_sift)} bits, QBER={qber:.2%}")

        if qber > qber_cutoff:
            if verbose: print("  > Discarded: QBER above cutoff.")
            continue   # throw away this round (possible Eve / noisy)

        collected.extend(a_sift)  # a_sift and b_sift match closely if QBER low
        if len(collected) >= target_bits:
            return collected[:target_bits], {"rounds_used": r, "qber_last": last_qber, "sifted_total": len(collected)}

    raise RuntimeError("Insufficient secure bits. Increase n_per_round/max_rounds or relax qber_cutoff.")

In [15]:
# @title 🧑‍🔬 Converting bits to bytes

def bits_to_bytes(bit_list):
    n = len(bit_list) - (len(bit_list) % 8)  # truncate to full bytes
    bit_list = bit_list[:n]
    by = bytes(int(''.join(map(str, bit_list[i:i+8])), 2) for i in range(0, n, 8))
    return by

def get_aes_key_from_bb84(bits=128, **bb84_kwargs):
    bit_list, meta = get_bb84_key(target_bits=bits, **bb84_kwargs)
    key = bits_to_bytes(bit_list)
    if len(key) != bits // 8:
        raise ValueError("Did not produce exact AES key length.")
    return key, meta

def get_des_key_from_bb84(bits=64, **bb84_kwargs):
    bit_list, meta = get_bb84_key(target_bits=bits, **bb84_kwargs)
    key = bits_to_bytes(bit_list)
    if len(key) != bits // 8:
        raise ValueError("Did not produce exact DES key length.")
    return key, meta

In [16]:
# @title 🔐 Encrypt AES and DES using quantum keys

def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return cipher.iv + ct

def aes_decrypt(ciphertext, key):
    iv, ct = ciphertext[:16], ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size).decode()

def des_encrypt(plaintext, key):
    cipher = DES.new(key, DES.MODE_CBC)
    ct = cipher.encrypt(pad(plaintext.encode(), DES.block_size))
    return cipher.iv + ct

def des_decrypt(ciphertext, key):
    iv, ct = ciphertext[:8], ciphertext[8:]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), DES.block_size).decode()

In [ ]:
# @title 🧪 Testing the things

# Toggle Eve to see keygen fail (or rounds discarded)
EVE = True   # set True to observe high QBER blocks

# Generate AES‑128 key via BB84 (will automatically run multiple rounds if needed)
aes_key, meta_aes = get_aes_key_from_bb84(
    bits=128, n_per_round=256, qber_cutoff=0.11, max_rounds=20, eve_present=EVE, verbose=True
)
print("\nAES key (hex):", aes_key.hex())
print("Meta:", meta_aes)

pt = input("Enter a plain text for testing the AES and DES encryption and decryption using Quantum Keys : ")
ct = aes_encrypt(pt, aes_key)
rt = aes_decrypt(ct, aes_key)
print("\nAES plaintext: ", pt)
print("\nAES key      : ", aes_key.hex())
print("\nAES Ciphered : ", ct.hex())
print("\nAES Decrypt  : ", rt)
print("AES roundtrip ok:", rt == pt)

# (Optional) DES demo
des_key, meta_des = get_des_key_from_bb84(
    bits=64, n_per_round=256, qber_cutoff=0.11, max_rounds=20, eve_present=EVE, verbose=False
)
print("\nDES key (hex):", des_key.hex())
ct2 = des_encrypt(pt, des_key)
rt2 = des_decrypt(ct2, des_key)
print("\nDES plaintext: ", pt)
print("\nDES key      : ", des_key.hex())
print("\nDES Ciphered : ", ct2.hex())
print("\nDES Decrypt  : ", rt2)
print("DES roundtrip ok:", rt2 == pt)

Round 1: sifted=137 bits, QBER=28.47%
  > Discarded: QBER above cutoff.
Round 2: sifted=121 bits, QBER=20.66%
  > Discarded: QBER above cutoff.
Round 3: sifted=135 bits, QBER=20.00%
  > Discarded: QBER above cutoff.
Round 4: sifted=147 bits, QBER=34.01%
  > Discarded: QBER above cutoff.
Round 5: sifted=117 bits, QBER=17.95%
  > Discarded: QBER above cutoff.
Round 6: sifted=132 bits, QBER=27.27%
  > Discarded: QBER above cutoff.
